In [ ]:
import urllib.request
import json

client_id = "w0bqIqbJDwSIbYWRQqWh"
#나의 인증키 
client_secret = "l_JzmWae7O"
#나의 비밀번호 
word = input("검색할 단어: ")
#input으로 검색할 단어를 입력받아 word에 저장 
sort_num = input("검색 결과 정렬 방법을 입력하세요(1:정확도순, 2:최신순)")
#input으로 정렬 방법을 sort_num에 저장 
str = ""
#정렬 방법을 저장할 변수 
if sort_num == "1":
    str = "sim"
#sort_num이 1이면 정렬 방법을 정확도순으로 
else:
    str = "date"
#sort_num이 2면 정렬 방법을 최신순으로 정렬한다.
encText = urllib.parse.quote(word)
#검색하고 싶은 단어를 인코딩 
total_count = 0
#총 검색한 단어의 수를 측정하는 변수 
import_number=0
#가져온 데이터 수를 나타내는 변수 
for start in range(1, 1001, 100):
#100개씩 총 1000개의 뉴스를 가져오기 위한 반복문 
    url = f"https://openapi.naver.com/v1/search/news?query={encText}&sort={str}&start={start}&display=100"
    # 쿼리스트링을 이용해 검색할 단어, 정렬 순서, 시작과 끝 개수를 입력 
    request = urllib.request.Request(url)
    #url 요청 추상화 객체 생성 
    request.add_header("X-Naver-Client-Id", client_id)
    # http 요청 객체에 인증키 추가 
    request.add_header("X-Naver-Client-Secret", client_secret)
    # http 요청 객체에 암호키  추가 
    response = urllib.request.urlopen(request)
    #request 객체의 url을 여는 함수 
    rescode = response.getcode()
    #http의 상태 코드 확인

    if rescode == 200:
      #응답 코드가 200이면 성공 
        response_body = response.read()
        #데이터를 바이트형으로 반환 
        data = json.loads(response_body.decode('utf-8'))
        #json 문자열을 python 객체로 변환 
        import_number=data['total']
        #읽어온 전체 데이터 수를 import_number에 저장 
        items = data['items']
        #기사들을 items에 저장 
        for item in items:
          #items에 있는 item들에 대해 
            print(item['title'])
            #제목만 출력 
            total_count += 1
            #개수에 1을 더한다. 
    else:
      #응답 코드가 200이 아니면 실패 
        print("Error Code:" + str(rescode))
        #에러 코드를 출력 후 종료 
        break
print(f"전체 검색: {total_count}")
#전체 검색 데이터수와 
print(f"가져온 데이터: {import_number}")
#가져온 데이터를 출력 

In [ ]:
import requests
import urllib.request
import json
print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
country=int(input("국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국 : 275)"))
nStartYear=int(input("데이터를 몇 년부터 수집할까요?: "))
nEndYear=int(input("데이터를 몇 년까지 수집할까요?: "))
#검색하고 싶은 국가코드, 수집 시작년도, 수집 마칠년도를 입력 받는다.
url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList'
#url을 string으로 저장 
result = []
#결과를 저장 
for year in range(nStartYear, nEndYear+1): 
#시작 년도부터 끝나는 년도까지 반복 
    for month in range(1, 13):
      #1월 부터 12월까지 
        yyyymm = "{0}{1:0>2}".format(str(year), str(month))
        #년도와 월을 string 형태로 저장 
        params ={'serviceKey' :'PfWt1sOBQGH5GQneFl07cSfFWZBqFWKIsBaux3YMFMGv3Z6IK4Y%2B31EiXfcKxZqbUjhiwAPU%2FJyrqf5nxtZRAw%3D%3D', 'YM' : yyyymm, 'NAT_CD' : str(country), 'ED_CD' : 'E', '_type' : 'json' }
        #params에 서비스키, 검색하고자하는 국가, 시작년도와 끝나는 년도등을 대입 
        response = requests.get(url, params=params)
        #get 방식 http 요청 
        jsonData = response.json()
        #json 형태로 변환 
        if (jsonData['response']['header']['resultMsg'] == 'OK'):
            natCd = jsonData['response']['body']['items']['item']['natCd']
            natName = jsonData['response']['body']['items']['item']['natKorNm']
            natName = natName.replace(' ', '')
            num = jsonData['response']['body']['items']['item']['num']
            result.append([natName, natCd, yyyymm, num])
          #json에서 원하는 데이터들을 추출 후, result에 추가 
import pandas as pd
columns=["입국자국가","국가코드","입국연월","입국자 수"]
result_df=pd.DataFrame(result,columns=columns)
#결과 값을 dataframe으로 저장 
import matplotlib.pyplot as plt
plt.rc('font', family = 'Malgun Gothic') 
result_df1 = result_df.drop(['입국자국가', '국가코드'], axis=1)
result_df1.plot(x='입국연월',y='입국자 수')
title = '%s 방한외래관광객 %s~%s' % (natName, nStartYear, nEndYear)
plt.title(title)
plt.xlabel("연월")
plt.ylabel("입국자 수")
plt.show()
#pyplot을 이용하여 result를 출력 